In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
# resample df_pos at the timestamps in df
MAX_ALLOWED_LAG_MS = 20

def add_pose_to_df(df, df_pos, max_allowed_lag_ms=MAX_ALLOWED_LAG_MS):
    for i, row in df.iterrows():
        timestamp = row.timestamp

        # most recent position timestamp
        if not len(df_pos[df_pos.timestamp <= timestamp]):
            df.loc[i, 'yaw_deg'] = None
            print('no position before first audio:', timestamp, df_pos.timestamp.values)
            continue
        pos_idx = df_pos[df_pos.timestamp <= timestamp].index[-1]
        lag = timestamp - df_pos.loc[pos_idx].timestamp
        if lag <= MAX_ALLOWED_LAG_MS:
            df.loc[i, 'yaw_deg'] = df_pos.loc[pos_idx].yaw_deg
            df.loc[i, 'dx'] = df_pos.loc[pos_idx].dx
            df.loc[i, 'dy'] = df_pos.loc[pos_idx].dy
        else:
            df.loc[i, 'yaw_deg'] = None
            print('too high time lag (in ms):', lag)

In [ ]:
from evaluate_data import read_df, get_spec

#exp_name = '2020_10_30_dynamic'; degree=0; start_idx = 0# correct mic positions, but only one degree
#exp_name = '2020_10_30_dynamic_test'; degree=0; start_idx=0 #degree=90; 
#exp_name = '2020_10_30_dynamic_move'; degree=0; start_idx=0
#exp_name = '2020_11_03_sweep_old'; degree=90; start_idx = 20


# TODO: clean the csv file instead of reading only from start_idx here. 
exp_name = '2020_11_03_sweep'; degree=90; start_idx = 20

params = dict(
    degree = degree,
    props = False,
    snr = False,
    motors = True,
    source = 'mono_linear',
    exp_name = exp_name
)

df, df_pos = read_df(**params)
df.head()
f_wav, t_wav, spectrogram_wav = get_spec(**params)

df = df.iloc[start_idx:]
index_start = df.iloc[0]['index']
df_pos = df_pos.loc[df_pos.index >= index_start]

add_pose_to_df(df, df_pos)

df.head()

In [ ]:
plt.figure()
plt.scatter(df_pos.timestamp.values, [1]*len(df_pos), label='df_pos')
plt.scatter(df.timestamp.values, [0]*len(df), label='df')
plt.legend()

start_pos = np.array([0, 0])
positions = np.empty([len(df), 2])

plt.figure()
i_row = 0
for i, row in df.iterrows():
    yaw_rad = row.yaw_deg / 180 * np.pi
    length = np.sqrt(row.dx**2 + row.dy**2) 
    new_pos = start_pos + length * np.array([np.cos(yaw_rad), np.sin(yaw_rad)])
    positions[i_row, :] = new_pos
    i_row += 1
    plt.scatter(row.timestamp/1000-df.timestamp.iloc[0]/1000, row.yaw_deg, color='C0')
plt.xlabel('time [s]')
plt.ylabel('yaw [deg]')
    
plt.figure()
plt.scatter(positions[:, 0], positions[:, 1])
plt.axis('equal')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')

In [ ]:
from audio_stack.beam_former import BeamFormer, normalize_rows, combine_rows
import progressbar

combination_n = 5
combination_method = "sum"
normalization_method = "none"
method = "mvdr"
bin_selection = "fixed"; selected_hz = [800]

# covariance averaging
#averaging = 'iir'
#alpha = 0.5 # set to 1 for no effect
#R = None

averaging = 'ma'
R_idx = 0; R_list = [[]] * combination_n

mic_positions = df.iloc[0].mic_positions
beam_former = BeamFormer(mic_positions=mic_positions)

if bin_selection == "fixed":
    selected_idx = [np.argmin(np.abs(df.iloc[0].frequencies - hz)) for hz in selected_hz]
    frequencies = df.iloc[0].frequencies[selected_idx] 
    beam_former.init_dynamic_estimate(frequencies, combination_n, combination_method, normalization_method)
    beam_former.init_multi_estimate(frequencies, combination_n)
else:
    # TODO(FD): implement good initialization
    pass

n_columns = len(df) #100
angles = beam_former.theta_scan
times = np.empty(n_columns)
raw_heatmap = np.zeros((len(angles), n_columns))
dynamic_heatmap = np.zeros((len(angles), n_columns))
multi_heatmap = np.zeros((len(angles), n_columns))
spectrogram = np.zeros((len(frequencies), mic_positions.shape[0], n_columns), dtype=np.complex)

i_col = 0

dynamic_spectrum = None
multi_spectrum = None

print('start at degrees:', df.yaw_deg.iloc[0])
with progressbar.ProgressBar(max_value=n_columns, redirect_stdout=True) as p:
    for __, row in df.iterrows():
        
        if bin_selection == "fixed":
            signals_f = row.signals_f[:, selected_idx] # n_mics x n_freqs
            freqs = row.frequencies[selected_idx]
        else:
            # TODO(FD): implement this
            pass

        orientation_deg = row.yaw_deg
        timestamp = row.timestamp
        
        R_new = beam_former.get_correlation(signals_f.T) # n_freqs x n_mics x n_mics
        if averaging == 'iir':
            if R is None:
                R = R_new
            else:
                R = alpha*R_new + (1-alpha)*R
        elif averaging == 'ma':
            R_list[R_idx] = R_new
            R_idx = (R_idx + 1) % combination_n
            R = np.mean([R_elem for R_elem in R_list if len(R_elem)], axis=0)
        
        if method == 'mvdr':
            raw_spectrum = beam_former.get_mvdr_spectrum(R, freqs)
        elif method == 'das':
            raw_spectrum = beam_former.get_das_spectrum(R, freqs)
        time_sec = row.audio_timestamp / 1e6
        
        if not pd.isna(row.yaw_deg):
            #### DYNAMIC
            beam_former.add_to_dynamic_estimates(raw_spectrum, orientation_deg=orientation_deg)
            dynamic_spectrum = beam_former.get_dynamic_estimate()
            dynamic_spectrum = combine_rows(dynamic_spectrum, combination_method, keepdims=True)
            dynamic_spectrum = normalize_rows(dynamic_spectrum, method="zero_to_one")

            #### MULTI_NEW
            beam_former.add_to_multi_estimate(signals_f.T, freqs, time_sec, -orientation_deg)
            multi_spectrum = beam_former.get_multi_estimate(method='mvdr')
            multi_spectrum = combine_rows(multi_spectrum, combination_method, keepdims=True)
            multi_spectrum = normalize_rows(multi_spectrum, method="zero_to_one")
        else:
            print('not updating dynamic and multi:', i_col)
                
        #### RAW
        raw_spectrum = combine_rows(raw_spectrum, combination_method, keepdims=True)
        raw_spectrum = normalize_rows(raw_spectrum, method="zero_to_one")

        raw_heatmap[:, i_col] = raw_spectrum.flatten()
        if dynamic_spectrum is not None:
            dynamic_heatmap[:, i_col] = dynamic_spectrum.flatten()
        if multi_spectrum is not None:
            multi_heatmap[:, i_col] = multi_spectrum.flatten()
        times[i_col] = time_sec
        spectrogram[:, :, i_col] = signals_f.T
        
        i_col += 1

        if i_col >= n_columns:
            break
            
        p.update(i_col)

In [ ]:
fig, axs = plt.subplots(1, 3)
fig.set_size_inches(15, 5)

times = range(raw_heatmap.shape[1])
angles = BeamFormer.theta_scan * 180 / np.pi

axs[0].pcolormesh(times, angles, raw_heatmap)
axs[0].set_title('raw heatmap')

axs[1].pcolormesh(times, angles, multi_heatmap)
axs[1].set_title('multi heatmap')

axs[2].pcolormesh(times, angles, dynamic_heatmap)
axs[2].set_title('dynamic heatmap')

# Plotting (can start here directly!)

In [ ]:
from audio_stack.beam_former import BeamFormer
plots = [
    dict(exp_name = '2020_10_30_dynamic_test', degree=0, motors=''),
    dict(exp_name = '2020_10_30_dynamic_test', degree=90, motors=''),
    dict(exp_name = '2020_10_30_dynamic_move', degree=0, motors='')
]

plots = [
    dict(exp_name = '2020_11_03_sweep', degree=90, motors=''),
    #dict(exp_name = '2020_11_03_sweep_old', degree=90, motors=''),
]

BeamFormer()

for plot_dict in plots:
    
    exp_name = plot_dict.get('exp_name')
    degree = plot_dict.get('degree')
    fname = f'DynamicAnalaysis_{exp_name}.pkl'
    df_results = pd.read_pickle(fname)
    print('read', fname)
    
    df_raw = df_results.loc[(df_results.exp_name==exp_name) & 
                               (df_results.degree==degree)]
    for (motors, combination_n, method, averaging), df_chosen in df_raw.groupby(['motors', 'combination_n', 'method', 'covariance_averaging']):
        
        assert len(df_chosen) == 1, len(df_chosen)
        
        df_chosen=df_chosen.iloc[0]
        
        if method == 'mvdr':
            continue

        plot_dict['motors'] = motors
        plot_dict['combination_n'] = combination_n
        plot_dict['method'] = method
        plot_dict['averaging'] = averaging

        times = np.arange(df_chosen.raw_heatmap.shape[1])
        angles = BeamFormer.theta_scan * 180 / np.pi

        fig, axs = plt.subplots(1, 3)
        fig.set_size_inches(15, 5)
        axs[0].pcolormesh(times, angles, df_chosen.raw_heatmap)
        axs[0].set_title('raw heatmap')

        axs[1].pcolormesh(times, angles, df_chosen.multi_heatmap)
        axs[1].set_title('multi heatmap')

        axs[2].pcolormesh(times, angles, df_chosen.dynamic_heatmap)
        axs[2].set_title('dynamic heatmap')

        axs[0].set_ylabel('angle [deg]')
        fig.suptitle(plot_dict)
        
    for motors, df_chosen in df_raw.groupby(['motors']):
        df_chosen = df_chosen.iloc[0]
        
        fig, ax = plt.subplots()
        fig.set_size_inches(15, 5)
        spec = df_chosen.spectrogram
        for mic_i in range(spec.shape[1]):
            ax.semilogy(times, np.abs(spec[0, mic_i, :]), label=f'mic{mic_i}')
        ax.semilogy(times, np.sum(np.abs(spec[0, :, :]), axis=0), label='total')
        ax.legend()
        fig.suptitle(f'motors: {motors}')
        
        continue
        fig, axs = plt.subplots(3, 2)
        fig.set_size_inches(15, 5)
        cond = df_chosen.condition_numbers
        ranks = df_chosen.ranks
        for i, label in enumerate(['raw', 'averaged', 'multi']):
            axs[i, 0].semilogy(times, cond[i, 0, :], label=label, color=f'C{i}')
            axs[i, 1].scatter(times+0.5*i, ranks[i, 0, :], label=label, color=f'C{i}')
            axs[i, 0].legend(loc='upper right')
            axs[i, 1].legend(loc='upper right')